In [ ]:
!pip install gradio

**Image Input**

In [ ]:
import cv2
import gradio as gr
import numpy as np

face1 = "/content/drive/MyDrive/Minor Project/opencv_face_detector.pbtxt"
face2 = "/content/drive/MyDrive/Minor Project/opencv_face_detector_uint8.pb"
age1 = "/content/drive/MyDrive/Minor Project/age_deploy.prototxt"
age2 = "/content/drive/MyDrive/Minor Project/age_net.caffemodel"
gen1 = "/content/drive/MyDrive/Minor Project/gender_deploy.prototxt"
gen2 = "/content/drive/MyDrive/Minor Project/gender_net.caffemodel"

MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)

# Face
face = cv2.dnn.readNet(face2, face1)
# age
age = cv2.dnn.readNetFromCaffe(age1, age2)
# gender
gen = cv2.dnn.readNetFromCaffe(gen1, gen2)

# Categories of distribution
la = ['(0, 2)', '(4, 6)', '(8, 12)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)']
lg = ['Male', 'Female']

def predict_age_gender(input_image):
    input_image = cv2.resize(input_image, (980, 980))
    fr_cv = np.array(input_image)
    fr_h, fr_w, _= fr_cv.shape
    blob = cv2.dnn.blobFromImage(fr_cv, 1.0, (300, 300),
                                [104, 117, 123], True, False)

    face.setInput(blob)
    detections = face.forward()
    faceBoxes = []
    for i in range(detections.shape[2]):
        confidence = detections[0, 0, i, 2]
        if confidence > 0.92:
            x1 = int(detections[0, 0, i, 3]*fr_w)
            y1 = int(detections[0, 0, i, 4]*fr_h)
            x2 = int(detections[0, 0, i, 5]*fr_w)
            y2 = int(detections[0, 0, i, 6]*fr_h)
            faceBoxes.append([x1, y1, x2, y2])
            cv2.rectangle(fr_cv, (x1, y1), (x2, y2),
                          (0, 350, 0), int(round(fr_h/250)), 8)

    if not faceBoxes:
        return fr_cv

    for faceBox in faceBoxes:
        face_img = fr_cv[max(0, faceBox[1] - 60):
                          min(faceBox[3] + 60, fr_h - 1),
                          max(0, faceBox[0] - 60):min(faceBox[2] + 60,
                                                       fr_w - 1)]

        blob = cv2.dnn.blobFromImage(
            face_img, 1.0, (227, 227), MODEL_MEAN_VALUES, swapRB=False)

        gen.setInput(blob)
        genderPreds = gen.forward()
        gender = lg[genderPreds[0].argmax()]

        age.setInput(blob)
        agePreds = age.forward()

        predictedAge = la[agePreds[0].argmax()]

        #age_conf_threshold = 0.99

        #if np.max(agePreds[0]) > age_conf_threshold:
            #age_index = np.argmax(agePreds)
            #predictedAge = la[age_index]

        cv2.putText(fr_cv,
                    f'{gender}, {predictedAge}',
                    (faceBox[0] + 20, faceBox[3] + 20),
                    cv2.FONT_ITALIC,
                    3.0,
                    (217, 0, 60),
                    3,
                    cv2.LINE_AA)

    return fr_cv, f'Gender: {gender}\nAge: {predictedAge}'

iface = gr.Interface(
    fn=predict_age_gender,
    inputs=gr.Image(label="Image Input"),
    outputs=['image','text'],
    title="Gender & Age Detection",
    description="Identifies Faces & Detects Gender & Age Acoordingly",

)

iface.launch()
